In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load data
train = pd.read_csv('Downloads/digit-recognizer/train.csv')
test = pd.read_csv('Downloads/digit-recognizer/test.csv')

# Split train data into features and labels
X = train.drop("label", axis=1).values
y = train["label"].values

# Normalize features
X = X.astype('float32') / 255.0
X_test_real = test.values.astype('float32') / 255.0  # test set for submission

# Reshape for CNN
X = X.reshape(-1, 28, 28, 1)
X_test_real = X_test_real.reshape(-1, 28, 28, 1)

# One-hot encode labels
y_cat = to_categorical(y, 10)

# Split training data into train/validation sets
X_train_img, X_val_img, y_train_cat, y_val_cat = train_test_split(
    X, y_cat, test_size=0.2, random_state=42
)

# Build CNN
cnn_model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history_cnn = cnn_model.fit(
    X_train_img, y_train_cat,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_img, y_val_cat),
    verbose=1
)


Epoch 1/20


E:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9121 - loss: 0.2814 - val_accuracy: 0.9793 - val_loss: 0.0667
Epoch 2/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9716 - loss: 0.0932 - val_accuracy: 0.9830 - val_loss: 0.0491
Epoch 3/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9797 - loss: 0.0691 - val_accuracy: 0.9887 - val_loss: 0.0377
Epoch 4/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9829 - loss: 0.0539 - val_accuracy: 0.9879 - val_loss: 0.0383
Epoch 5/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9852 - loss: 0.0468 - val_accuracy: 0.9882 - val_loss: 0.0339
Epoch 6/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9873 - loss: 0.0416 - val_accuracy: 0.9899 - val_loss: 0.0310
Epoch 7/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9884 - loss: 0.0348 - val_accuracy: 0.9895 - val_loss: 0.0320
Epoch 8/20
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9897 - loss: 0.0324 - val_accurac

In [8]:
acc_cnn=cnn_model.evaluate(X_val_img,y_val_cat,verbose=0)[1]

In [9]:
acc_cnn

0.9921428561210632

In [13]:
# 1️⃣ Predict probabilities for each class
pred_probs = cnn_model.predict(X_test_real)

# 2️⃣ Convert probabilities to class labels
pred_labels = np.argmax(pred_probs, axis=1)

# 3️⃣ Create submission DataFrame
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(pred_labels) + 1),
    "Label": pred_labels
})

# 4️⃣ Save to CSV
submission.to_csv("submission.csv", index=False)

print("Submission file created: submission.csv")

875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Submission file created: submission.csv
